In [2]:
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying, flatten_dict_column
import pandas as pd
from typing import List

In [9]:
import json
import os

In [1]:
config = {
    "tableau-server" : {
        "server" : 'http://10.100.252.218/',
        "api_version" : "3.19",
        "personal_access_token_name" : "rest_token",
        "personal_access_token_secret" : "zL6zdQRgS9WQYH24aGR4JQ==:7yDTI0BCA6XlNxOm24YDN6S3B6673Nyn",
        "site_name" : "890Portal",
        "site_url" : "890Portal"
    }
}

In [4]:
conn = TableauServerConnection(config_json=config,env="tableau-server")

In [5]:
response = conn.sign_in()

In [6]:
response

<Response [200]>

In [154]:
views_df = querying.get_views_dataframe(conn)

In [155]:
views_df = flatten_dict_column(views_df,keys=["name","id"],col_name="workbook")

In [160]:
dashid = '4c29991b-0b11-4243-8a1b-650f52e236d5'

In [161]:
view_png = conn.query_view_image(view_id=dashid)

In [162]:
with open('Dashboard.png','wb') as v:
    v.write(view_png.content)

In [85]:
def add_workbook_name_to_csv(csv_path):
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Define a function to extract the 'name' from the project JSON
    def extract_project_name(project_json):
        try:
            project_dict = json.loads(project_json.replace("'", '"'))
            return project_dict.get('name', '')
        except json.JSONDecodeError:
            return ''

    # Apply the function to the 'project' column to create the 'workbook_name' column
    df['workbook_name'] = df['project'].apply(extract_project_name)

    # Save the modified DataFrame back to the CSV file
    df.to_csv(csv_path, index=False)
    print(f"Successfully added 'workbook_name' column to {csv_path}")

# Example usage
add_workbook_name_to_csv('Views.csv')

Successfully added 'workbook_name' column to Views.csv


In [62]:
views_df.to_csv("Views.csv")

In [139]:
check_view_id = "a28720b9-b2af-42ef-9c7d-a98cd9d1f129" 

In [140]:
check_view_df = querying.get_view_data_dataframe(conn,view_id=check_view_id)

In [148]:
fn = 'RuralUrbanDistribution2019.csv'

In [149]:
check_view_df.to_csv(fn)

In [77]:
cc_df = querying.get_sites_dataframe(conn)

In [79]:
cc_df.to_csv("Sites.csv")

In [80]:
wbs = querying.get_workbooks_dataframe(conn)

In [82]:
wbs.to_csv("Workbook.csv")

In [70]:
project_folder = "890Portal"
if not os.path.exists(project_folder):
    os.makedirs(project_folder)
    
def query_and_save_views(csv_path, conn):
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        check_view_id = row['id']
        name = row['name']
        content_url = row['contentUrl']

        # Extract page_name from contentUrl
        page_name = content_url.split('/')[0]

        try:
            # Query the data
            check_view_df = querying.get_view_data_dataframe(conn, view_id=check_view_id)

            # Create the folder if it doesn't exist
            if not os.path.exists(page_name):
                os.makedirs(page_name)

            # Define the file path
            file_path = os.path.join(os.path.join(project_folder,page_name,f"{name}.csv"))

            # Save the DataFrame to a CSV file
            check_view_df.to_csv(file_path, index=False)

            print(f"Successfully saved data for view ID {check_view_id} to {file_path}")
        except Exception as e:
            print(f"Error querying or saving data for view ID {check_view_id}: {e}")


In [71]:
query_and_save_views('Views.csv', conn)

Successfully saved data for view ID 6047b331-14cc-485b-a6a8-eeebdfd7f03a to CMO_Dashboard-WIP\Reach.csv
Successfully saved data for view ID c61b802a-8a5d-49fe-a6f9-91eb6e4ceea3 to CMO_Dashboard-WIP\Promotion Effect.csv
Successfully saved data for view ID 608163b1-d010-4514-9277-f693d8c64b26 to CMO_Dashboard-WIP\Conversion.csv
Successfully saved data for view ID 25d485af-bfc4-4451-9fbb-661888b37ff7 to CMO_Dashboard\Dashboard 1.csv
Successfully saved data for view ID cbcc0b15-9c83-40a6-aea0-6768593ad506 to CMO_Dashboard\Reach.csv
Successfully saved data for view ID 2d15e4db-ea02-46ea-ae1c-4b23f43cc890 to CMO_Dashboard\Promotion Effect.csv
Successfully saved data for view ID a31a6cc1-104f-4351-bd1a-df62814990b1 to CMO_Dashboard\Conversion.csv
Successfully saved data for view ID 8cbc3450-ad21-461e-802e-ac82bfac5126 to CMO_Dashboard\Content.csv
Successfully saved data for view ID db5141d1-6d0d-4e6b-88cb-0b4af20be461 to ConsolidatedCookbook\Cycle Times on Insurance Claim.csv
Error querying o

Successfully saved data for view ID 73dc69d9-1dda-49da-9517-aeb5cdca352b to BacktestAssetAllocationDashboard\Rolling Returns Analysis.csv
Successfully saved data for view ID 8f6f1972-998a-40bf-b618-c146ce7c28ab to BacktestAssetAllocationDashboard\Rolling Returns Trend.csv
Successfully saved data for view ID 4a6577c2-2728-4a9c-a4b4-98df56376e31 to BacktestAssetAllocationDashboardv3\Asset Allocation.csv
Successfully saved data for view ID a7122bb9-ea8d-48c0-b87d-f1adfed83eb1 to BacktestAssetAllocationDashboardv3\Portfolio Performance.csv
Successfully saved data for view ID 98a4f7be-e453-4192-b317-1b935b6215f9 to BacktestAssetAllocationDashboardv3\Portfolio Growth and Returns.csv
Successfully saved data for view ID 1a283348-c210-479b-ab8f-b9537702a1db to BacktestAssetAllocationDashboardv3\Annual Returns and Balance Data.csv
Successfully saved data for view ID 76bf1e9e-c6af-4a1d-b58a-1c744f53ab16 to BacktestAssetAllocationDashboardv3\Risk and Return Metrics.csv
Successfully saved data for 

In [87]:
dashboard_views_df = views_df[views_df['sheetType'] == 'dashboard']

In [89]:
dashboard_views_df.to_csv("Dashboard.csv")

In [91]:
from tableau_api_lib.utils.querying import get_views_dataframe, get_view_data_dataframe

In [102]:
zz = get_view_data_dataframe(conn, view_id='25d485af-bfc4-4451-9fbb-661888b37ff7')

In [103]:
zz

,Asset_Brwse,Date,Active User
0,Asset1,10,23.882400
1,Asset1,11,34.155075
2,Asset1,12,68.092027
3,Asset1,13,29.853181
4,Asset1,14,11.467845
5,Asset1,15,67.207830
6,Asset2,10,34.830329
7,Asset2,11,60.662333
8,Asset2,12,74.880070
9,Asset2,13,34.256938


In [107]:
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying, flatten_dict_column
import pandas as pd

In [105]:
wbs.head(1)

,project,location,owner,tags,dataAccelerationConfig,id,name,description,contentUrl,webpageUrl,showTabs,size,createdAt,updatedAt,encryptExtracts,defaultViewId
0,"{'id': 'c02fb59d-885b-4055-bab8-3f1d2e5ac520',...","{'id': 'c02fb59d-885b-4055-bab8-3f1d2e5ac520',...","{'id': '36eebeb5-be56-4cdc-a79c-d65c18ea2f8c',...",{},{'accelerationEnabled': False},d7586af6-a1df-4f6e-8ccf-1399d76430e9,CMO_Dashboard-WIP,,CMO_Dashboard-WIP,http://EC2AMAZ-IST7KPQ/#/site/890Portal/workbo...,false,1,2020-02-06T17:33:43Z,2020-02-08T08:47:24Z,false,6047b331-14cc-485b-a6a8-eeebdfd7f03a


In [110]:
wb_df = flatten_dict_column(df=wbs,keys=["id","name"],col_name="project")

In [112]:
wb_df.head(1)

,location,owner,tags,dataAccelerationConfig,id,name,description,contentUrl,webpageUrl,showTabs,size,createdAt,updatedAt,encryptExtracts,defaultViewId,project_id,project_name
0,"{'id': 'c02fb59d-885b-4055-bab8-3f1d2e5ac520',...","{'id': '36eebeb5-be56-4cdc-a79c-d65c18ea2f8c',...",{},{'accelerationEnabled': False},d7586af6-a1df-4f6e-8ccf-1399d76430e9,CMO_Dashboard-WIP,,CMO_Dashboard-WIP,http://EC2AMAZ-IST7KPQ/#/site/890Portal/workbo...,false,1,2020-02-06T17:33:43Z,2020-02-08T08:47:24Z,false,6047b331-14cc-485b-a6a8-eeebdfd7f03a,c02fb59d-885b-4055-bab8-3f1d2e5ac520,Default


In [133]:
target_wb_df = (
    wb_df
    .where(wb_df['project_name'] == 'Consolidated Cookbook')
    .where(wb_df['name'] == 'Comparison of Financial Metrics')
    .dropna()
)

In [134]:
target_wb_df

,location,owner,tags,dataAccelerationConfig,id,name,description,contentUrl,webpageUrl,showTabs,size,createdAt,updatedAt,encryptExtracts,defaultViewId,project_id,project_name


In [120]:
target_view_df = querying.get_views_for_workbook_dataframe(
    conn, workbook_id = target_wb_df.get('id').to_list().pop()
)

In [121]:
target_view_df

,tags,id,name,contentUrl,createdAt,updatedAt,viewUrlName
0,{},25d485af-bfc4-4451-9fbb-661888b37ff7,Dashboard 1,CMO_Dashboard/sheets/Dashboard1,2020-02-08T08:45:16Z,2020-02-08T08:49:59Z,Dashboard1
1,{},cbcc0b15-9c83-40a6-aea0-6768593ad506,Reach,CMO_Dashboard/sheets/Reach,2020-02-08T08:45:16Z,2020-02-08T08:49:59Z,Reach
2,{},2d15e4db-ea02-46ea-ae1c-4b23f43cc890,Promotion Effect,CMO_Dashboard/sheets/PromotionEffect,2020-02-08T08:45:16Z,2020-02-08T08:49:59Z,PromotionEffect
3,{},a31a6cc1-104f-4351-bd1a-df62814990b1,Conversion,CMO_Dashboard/sheets/Conversion,2020-02-08T08:45:16Z,2020-02-08T08:49:59Z,Conversion
4,{},8cbc3450-ad21-461e-802e-ac82bfac5126,Content,CMO_Dashboard/sheets/Content,2020-02-08T08:45:16Z,2020-02-08T08:49:59Z,Content


In [123]:
dash_id = (
    target_view_df.where(target_view_df["name"] == "Dashboard 1").dropna().get("id").to_list().pop()
)

In [124]:
dash_id

'25d485af-bfc4-4451-9fbb-661888b37ff7'

In [125]:
resp = conn.query_view_data(view_id=dash_id)

In [126]:
with open("dashboard_download_attempt.csv","wb") as file:
    file.write(resp.content)

In [131]:
dash_name = "Comparison of Financial Metrics"

In [132]:
for index, row in target_view_df.iterrows():
    if row["name"] != dash_name:
        print(f'Creating CSV for {row["name"]}...')
        resp = conn.query_view_data(view_id=row["id"])
        with open(f"{row['name']}.csv","wb") as file:
            file.write(resp.content)

Creating CSV for Dashboard 1...
Creating CSV for Reach...
Creating CSV for Promotion Effect...
Creating CSV for Conversion...
Creating CSV for Content...


In [7]:
ROOT_PATH = r'C:\Users\NITINS\OneDrive - Capgemini\CAPGEMINI\PROJECT\GEN AI\report-usecase\tableau\tableau-auth\pat\890Portal'

In [10]:
portal_viewdf = pd.read_csv(os.path.join(ROOT_PATH,'890Portal_Views.csv'))

In [11]:
portal_dashdf = portal_viewdf[portal_viewdf['sheetType'] == 'dashboard']

In [12]:
portal_dashdf = portal_dashdf[['name','id','contentUrl','sheetType','viewUrlName','workbook_name','workbook_id']]

In [14]:
portal_dashdf.to_csv(os.path.join(ROOT_PATH,'890Portal_Dashboards.csv'))

In [27]:
os.makedirs(os.path.join(ROOT_PATH,'Dashboard_images'), exist_ok=True)

In [25]:
import re

In [183]:
for index, row in portal_dashdf.iterrows():
    dashid = row['id']
    print(f'Dash id : {dashid}')

Dash id : 6047b331-14cc-485b-a6a8-eeebdfd7f03a
Dash id : c61b802a-8a5d-49fe-a6f9-91eb6e4ceea3
Dash id : 608163b1-d010-4514-9277-f693d8c64b26
Dash id : 25d485af-bfc4-4451-9fbb-661888b37ff7
Dash id : cbcc0b15-9c83-40a6-aea0-6768593ad506
Dash id : 2d15e4db-ea02-46ea-ae1c-4b23f43cc890
Dash id : a31a6cc1-104f-4351-bd1a-df62814990b1
Dash id : 8cbc3450-ad21-461e-802e-ac82bfac5126
Dash id : 4c29991b-0b11-4243-8a1b-650f52e236d5
Dash id : 2d0e3014-b2e0-4d3c-a07e-a0f2cb3a2b59
Dash id : 1f72d65f-4166-44ae-907f-bb783a1531a1
Dash id : 5111ec49-5d6e-4102-ba0a-4bdd2d4bd101
Dash id : c86451f1-daf8-4fdd-b19b-5b11bf7c036f
Dash id : a93ce1c5-c0d7-40b3-a24d-415ada65467d
Dash id : 26144275-48db-425a-871a-cc98451f7269
Dash id : f3772e7e-817b-4795-9ae3-dbbf609f3e8a
Dash id : 3809bb07-3690-4c26-bc46-aed8bc6afb65
Dash id : b43be72e-3b7c-478b-957a-ce51bb4a204b
Dash id : 1e2e99fd-6a97-4b02-a05b-9d8cc941fec5
Dash id : 059e85ac-30d3-4ac5-b246-414dda975f6e
Dash id : dcede592-e082-4114-9989-a47d6d2ff7bc
Dash id : 2e0

In [35]:
# dashid = '25d485af-bfc4-4451-9fbb-661888b37ff7'
dashid = '6047b331-14cc-485b-a6a8-eeebdfd7f03a'
view_png = conn.query_view_image(view_id=dashid)
with open('check_sample2.png','wb') as v:
    v.write(view_png.content)

In [37]:
portal_dashdf.head(1)

,name,id,contentUrl,sheetType,viewUrlName,workbook_name,workbook_id
0,Reach,6047b331-14cc-485b-a6a8-eeebdfd7f03a,CMO_Dashboard-WIP/sheets/Reach,dashboard,Reach,CMO_Dashboard-WIP,d7586af6-a1df-4f6e-8ccf-1399d76430e9


In [40]:
for index, row in portal_dashdf.iterrows():
    dashid = row['id']  # No need to add extra quotes
    print(f"Dashid : {dashid}")  # To verify the dashid is correctly formatted
    name = row['name']
    view_url_name = row['viewUrlName']
    
    # Check for special characters in the 'name' and replace the filename if necessary
    if re.search(r'[\\/]', name):
        filename = view_url_name + '.png'
    else:
        filename = name + '.png'
    
    # Full path to save the image
#     filepath = os.path.join(output_dir, filename)
    
    # Extract the view image using Tableau API
    print(f"Trying to extract the image for {dashid}")
    view_png = conn.query_view_image(view_id=dashid)
    
    # Save the image with the appropriate filename
    with open(os.path.join(ROOT_PATH,'Dashboard_images',filename), 'wb') as v:
        v.write(view_png.content)
    print(f'The Dashboard image for {filename} is extracted')


Dashid : 6047b331-14cc-485b-a6a8-eeebdfd7f03a
Trying to extract the image for 6047b331-14cc-485b-a6a8-eeebdfd7f03a
The Dashboard image for Reach.png is extracted
Dashid : c61b802a-8a5d-49fe-a6f9-91eb6e4ceea3
Trying to extract the image for c61b802a-8a5d-49fe-a6f9-91eb6e4ceea3
The Dashboard image for Promotion Effect.png is extracted
Dashid : 608163b1-d010-4514-9277-f693d8c64b26
Trying to extract the image for 608163b1-d010-4514-9277-f693d8c64b26
The Dashboard image for Conversion.png is extracted
Dashid : 25d485af-bfc4-4451-9fbb-661888b37ff7
Trying to extract the image for 25d485af-bfc4-4451-9fbb-661888b37ff7
The Dashboard image for Dashboard 1.png is extracted
Dashid : cbcc0b15-9c83-40a6-aea0-6768593ad506
Trying to extract the image for cbcc0b15-9c83-40a6-aea0-6768593ad506
The Dashboard image for Reach.png is extracted
Dashid : 2d15e4db-ea02-46ea-ae1c-4b23f43cc890
Trying to extract the image for 2d15e4db-ea02-46ea-ae1c-4b23f43cc890
The Dashboard image for Promotion Effect.png is extra

The Dashboard image for Asset Allocation.png is extracted
Dashid : 38287d99-8f79-4d79-a601-7d74b6ba3039
Trying to extract the image for 38287d99-8f79-4d79-a601-7d74b6ba3039
The Dashboard image for Portfolio Performance.png is extracted
Dashid : a9aacc3d-d5fd-4e96-957c-28752966d05d
Trying to extract the image for a9aacc3d-d5fd-4e96-957c-28752966d05d
The Dashboard image for Portfolio Growth and Returns.png is extracted
Dashid : 4feb11fd-c36d-4a65-aeec-11ab6ebd151d
Trying to extract the image for 4feb11fd-c36d-4a65-aeec-11ab6ebd151d
The Dashboard image for Annual Returns and Balance Data.png is extracted
Dashid : 902a5cd3-f337-4b2f-8a9c-ba4261194ce3
Trying to extract the image for 902a5cd3-f337-4b2f-8a9c-ba4261194ce3
The Dashboard image for Risk and Return Metrics.png is extracted
Dashid : db6ea5ed-c199-4e10-a6d1-062ed4ce449a
Trying to extract the image for db6ea5ed-c199-4e10-a6d1-062ed4ce449a
The Dashboard image for Drawdowns.png is extracted
Dashid : 34a26b80-9d0e-44c5-8245-f730edefa0db

In [33]:
for index, row in portal_dashdf.iterrows():
    dashid = f"'{row['id']}'"
    print(f"Dashid : {dashid}")
    name = row['name']
    view_url_name = row['viewUrlName']
    
    # Check for special characters in the 'name' and replace the filename if necessary
    if re.search(r'[\\/]', name):
        filename = view_url_name + '.png'
    else:
        filename = name + '.png'
    
    # Extract the view image using Tableau API
    print(f"Trying to extract the image for {dashid}")
    view_png = conn.query_view_image(view_id=dashid)
    
    # Save the image with the appropriate filename
    with open(os.path.join(ROOT_PATH,'Dashboard_images',filename), 'wb') as v:
        v.write(view_png.content)
    print(f'The Dashboard image for {filename} is extracted')

Dashid : '6047b331-14cc-485b-a6a8-eeebdfd7f03a'
Trying to extract the image for '6047b331-14cc-485b-a6a8-eeebdfd7f03a'
Dashid : 'c61b802a-8a5d-49fe-a6f9-91eb6e4ceea3'
Trying to extract the image for 'c61b802a-8a5d-49fe-a6f9-91eb6e4ceea3'
Dashid : '608163b1-d010-4514-9277-f693d8c64b26'
Trying to extract the image for '608163b1-d010-4514-9277-f693d8c64b26'
Dashid : '25d485af-bfc4-4451-9fbb-661888b37ff7'
Trying to extract the image for '25d485af-bfc4-4451-9fbb-661888b37ff7'
Dashid : 'cbcc0b15-9c83-40a6-aea0-6768593ad506'
Trying to extract the image for 'cbcc0b15-9c83-40a6-aea0-6768593ad506'
Dashid : '2d15e4db-ea02-46ea-ae1c-4b23f43cc890'
Trying to extract the image for '2d15e4db-ea02-46ea-ae1c-4b23f43cc890'
Dashid : 'a31a6cc1-104f-4351-bd1a-df62814990b1'
Trying to extract the image for 'a31a6cc1-104f-4351-bd1a-df62814990b1'
Dashid : '8cbc3450-ad21-461e-802e-ac82bfac5126'
Trying to extract the image for '8cbc3450-ad21-461e-802e-ac82bfac5126'
Dashid : '4c29991b-0b11-4243-8a1b-650f52e236d5'


In [ ]:
dashid = 'fjslkchffbcxdhnsgkj'
view_png = conn.query_view_image(view_id=dashid)
with open('Dashboard.png','wb') as v:
    v.write(view_png.content)